# 2주차 기본과제: 주어진 문장에서 나올 다음 단어를 예측하는 모델 구현

last word prediction: token list가 주어졌을 때, 다음으로 오는 token을 예측하는 task

In [1]:
%pip install datasets sacremoses


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Users/gimga-eun/.pyenv/versions/3.12.9/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


데이터셋으로 IMDB를 사용한다.
IMDB는 영화 리뷰 데이터셋으로, 각 리뷰는 긍정(1) 또는 부정(0)으로 분류되어 있다.

데이터 로더를 생성한다.
문장의 마지막 토큰을 레이블로, 마지막 토큰을 제외한 토큰들을 텍스트로 사용한다.

In [2]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)
from torch.nn.utils.rnn import pad_sequence

train_ds = load_dataset("stanfordnlp/imdb", split="train")
test_ds = load_dataset("stanfordnlp/imdb", split="test")

tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')

def collate_fn(batch):
  # 각 문장의 최대 길이를 400으로 제한
  max_len = 400
  texts, labels = [], []
  for row in batch:
    # 문장의 마지막 토큰을 레이블로 사용
    # 특수 토큰인 마지막 토큰과 .,? 등 문장을 종결하는 기호를 제외하기 위해 -3를 사용
    labels.append(tokenizer(row['text'], truncation=True, max_length=max_len).input_ids[-3])
    # labels 앞의 토큰들을 텍스트로 사용
    texts.append(torch.LongTensor(tokenizer(row['text'], truncation=True, max_length=max_len).input_ids[:-3]))

  # 패딩 추가
  texts = pad_sequence(texts, batch_first=True, padding_value=tokenizer.pad_token_id)
  # 레이블을 텐서로 변환
  labels = torch.LongTensor(labels)

  return texts, labels


train_loader = DataLoader(
    train_ds, batch_size=64, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    test_ds, batch_size=64, shuffle=False, collate_fn=collate_fn
)

/Users/gimga-eun/.pyenv/versions/3.12.9/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using cache found in /Users/gimga-eun/.cache/torch/hub/huggingface_pytorch-transformers_main


## Self-attention

self-attention은 입력 시퀀스의 각 위치가 다른 모든 위치와 어떻게 관련되어 있는지 계산한다.
이를 통해 문맥을 고려한 표현을 학습할 수 있다.

In [3]:
from torch import nn
from math import sqrt


class SelfAttention(nn.Module):
  def __init__(self, input_dim, d_model):
    super().__init__()

    self.input_dim = input_dim
    self.d_model = d_model

    # Query, Key, Value 백터를 만들기 위한 가중치 행렬값 계산
    self.wq = nn.Linear(input_dim, d_model)
    self.wk = nn.Linear(input_dim, d_model)
    self.wv = nn.Linear(input_dim, d_model)
    self.dense = nn.Linear(d_model, d_model)

    # softmax를 사용하여 각 단어가 다른 모든 단어와 어떻게 관련되어 있는지를 0~1 사이의 확률로 표현한다.
    self.softmax = nn.Softmax(dim=-1)

  def forward(self, x, mask):
    q, k, v = self.wq(x), self.wk(x), self.wv(x)
    score = torch.matmul(q, k.transpose(-1, -2)) # (B, S, D) * (B, D, S) = (B, S, S)
    # 스케일링을 통해 값이 score가 너무 커져서 softmax 함수에서 기울기가 매우 작아지는 문제(vanishing gradient)를 방지한다.
    score = score / sqrt(self.d_model)

    # 패딩 토큰에 대한 정보를 무시하기 위해 mask를 사용한다.
    if mask is not None:
      score = score + (mask * -1e9)

    # softmax 적용하여 어텐션 가중치 생성
    score = self.softmax(score)

    # 어텐션 가중치와 value를 곱하여 새로운 representation을 생성
    result = torch.matmul(score, v)

    # 선형 변환 레이어를 통과하여 최종 출력을 생성
    result = self.dense(result)

    return result

Self-attention과 Feed-Forward Network를 사용해 Transformer layer를 구현한다.

Feed-Forward Network는 MLP의 한 종류로, attention이 찾은 관계를 더 깊게 처리하고, 특징 추출을 강화하는 역할을 한다.
2개의 선형 레이어로 구성되어 있으며, ReLU 활성화 함수를 사용하여 비선형성을 추가한다.

In [4]:
class TransformerLayer(nn.Module):
  def __init__(self, input_dim, d_model, dff):
    super().__init__()

    self.input_dim = input_dim  # 입력 데이터의 차원
    self.d_model = d_model      # 모델 내부에서 사용할 차원
    self.dff = dff              # Feed-Forward Network의 은닉층 크기

    # Self-Attention 레이어
    self.sa = SelfAttention(input_dim, d_model)

    # Feed-Forward Network (FFN) 레이어
    self.ffn = nn.Sequential(
      nn.Linear(d_model, dff),
      nn.ReLU(),
      nn.Linear(dff, d_model)
    )

  def forward(self, x, mask):
    x = self.sa(x, mask)
    x = self.ffn(x)

    return x

## Positional encoding

각 위치마다 고유한 패턴을 생성하고 sin과 cos 함수를 사용하여 상대적 위치 정보를 표현한다.
transformer 모델에 순서 정보를 제공한다.


In [5]:
import numpy as np

def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, None], np.arange(d_model)[None, :], d_model)
    # 짝수 부분
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    # 홀수 부분
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    # 차원 추가해서 형태 맞추기
    pos_encoding = angle_rads[None, ...]

    return torch.FloatTensor(pos_encoding)


max_len = 400
print(positional_encoding(max_len, 256).shape)

torch.Size([1, 400, 256])


TransformerLayer와 positional encoding을 결합해 TextClassifier를 구현한다.

In [6]:
class TextClassifier(nn.Module):
  
  def __init__(self, vocab_size, d_model, n_layers, dff):
    super().__init__()

    # vocab_size: 어휘 사전의 크기
    # d_model: 모델의 임베딩 차원
    # n_layers: 트랜스포머 레이어의 수
    # dff: Feed-Forward Network의 은닉층 차원
    self.vocab_size = vocab_size
    self.d_model = d_model
    self.n_layers = n_layers
    self.dff = dff

    # word embedding을 통해 단어를 고정된 크기의 백터로 변환한다.
    # 텍스트를 숫자로 표현하여 모델이 처리할 수 있게 한다.
    # 비슷한 의미의 단어는 비슷한 백터를 가지므로 의미적 유사성을 표현할 수 있다.
    self.embedding = nn.Embedding(vocab_size, d_model)

    # positional encoding을 통해 단어의 위치 정보를 표현한다.
    # 모델이 순서 정보를 고려할 수 있도록 한다.
    self.pos_encoding = nn.parameter.Parameter(positional_encoding(max_len, d_model), requires_grad=False)

    # 여러 개의 transformer layer를 쌓아서 모델의 깊이를 증가시킨다.
    self.layers = nn.ModuleList([TransformerLayer(d_model, d_model, dff) for _ in range(n_layers)])

    # 최종 분류 레이어
    # 입력 토큰의 특징을 vocabulary 크기의 벡터로 변환하여 다음 토큰을 예측
    self.classification = nn.Linear(d_model, len(tokenizer))

  def forward(self, x):
    # 패딩 토큰에 대한 정보를 무시하기 위해 mask를 사용한다.
    mask = (x == tokenizer.pad_token_id)
    mask = mask[:, None, :]

    # 문장의 길이
    seq_len = x.shape[1]

    # 단어 임베딩
    x = self.embedding(x)
    # 스케일링을 통해 값이 너무 커지는 것을 방지
    x = x * sqrt(self.d_model)
    # 위치 인코딩 추가
    x = x + self.pos_encoding[:, :seq_len]

    for layer in self.layers:
      x = layer(x, mask)

    # 마지막 토큰을 선택하고, 이 토큰 다음에 올 토큰을 예측
    x = x[:, -1]
    x = self.classification(x)

    return x


model = TextClassifier(len(tokenizer), 32, 2, 32)

## 학습

In [7]:
from torch.optim import Adam

# mac에 GPU를 사용하기 위한 설정
device = torch.device("mps")

lr = 0.001
model = model.to("mps")
# 분류 문제이므로 CrossEntropyLoss 사용
loss_fn = nn.CrossEntropyLoss()

optimizer = Adam(model.parameters(), lr=lr)

In [8]:
import numpy as np
import matplotlib.pyplot as plt

def accuracy(model, dataloader):
  cnt = 0
  acc = 0

  for data in dataloader:
    inputs, labels = data
    inputs, labels = inputs.to("mps"), labels.to("mps")

    preds = model(inputs)
    preds = torch.argmax(preds, dim=-1)

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt

In [9]:
n_epochs = 50

for epoch in range(n_epochs):
  total_loss = 0.
  model.train()
  for data in train_loader:
    model.zero_grad()
    inputs, labels = data
    inputs, labels = inputs.to("mps"), labels.to("mps")

    preds = model(inputs)
    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

  with torch.no_grad():
    model.eval()
    train_acc = accuracy(model, train_loader)
    test_acc = accuracy(model, test_loader)
    print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

Epoch   0 | Train Loss: 3017.768720149994
=========> Train acc: 0.039 | Test acc: 0.039
Epoch   1 | Train Loss: 2692.847065925598
=========> Train acc: 0.039 | Test acc: 0.039
Epoch   2 | Train Loss: 2663.41614818573
=========> Train acc: 0.036 | Test acc: 0.034
Epoch   3 | Train Loss: 2650.5880579948425
=========> Train acc: 0.039 | Test acc: 0.039
Epoch   4 | Train Loss: 2643.3305978775024
=========> Train acc: 0.036 | Test acc: 0.034
Epoch   5 | Train Loss: 2638.2893743515015
=========> Train acc: 0.041 | Test acc: 0.041
Epoch   6 | Train Loss: 2632.3808059692383
=========> Train acc: 0.037 | Test acc: 0.038
Epoch   7 | Train Loss: 2619.501808166504
=========> Train acc: 0.044 | Test acc: 0.040
Epoch   8 | Train Loss: 2582.4987745285034
=========> Train acc: 0.048 | Test acc: 0.045
Epoch   9 | Train Loss: 2549.124405860901
=========> Train acc: 0.058 | Test acc: 0.054
Epoch  10 | Train Loss: 2525.111057281494
=========> Train acc: 0.062 | Test acc: 0.057
Epoch  11 | Train Loss: 2500